In [1]:
# Autocompletion
%config Completer.use_jedi = False

# Autoreload  
%load_ext autoreload
%autoreload 2

In [2]:
# Autoreload
import sys
from typing import List

sys.path.append('../')

import numpy as np
from IPython.display import Audio, display


import nltk
import torch
import numpy as np
import datasets
import pandas as pd
import transformers

from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, T5ForConditionalGeneration, AutoModelForSeq2SeqLM

In [3]:
import llmsearch

Monkey Patching .generate function of `transformers` library


In [4]:
device = "cpu"

if torch.backends.mps.is_built() and torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"

from llmsearch.utils.mem_utils import gc_cuda

print(f"Device - {device}")

def beep(duration = 1, frequency=440, rhythm=1):
    sample_rate = 44100  # Standard audio sample rate
    t = np.linspace(0, duration, int(duration * sample_rate), endpoint=False)
    audio_data = np.sin(2*np.pi*frequency*t)  # Generate a sine wave
    audio_data *= np.where(np.arange(len(audio_data)) % rhythm == 0, 1, 0)  # Apply rhythm
    display(Audio(audio_data, rate=sample_rate, autoplay=True))

Device - mps


In [5]:
dataset = datasets.load_dataset("samsum")

Found cached dataset samsum (/Users/praful932/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
sample_size = 1
samples_to_tune_on = datasets.Dataset.from_dict(dataset["train"][:sample_size])
samples_to_tune_on = samples_to_tune_on.rename_columns(column_mapping = {'dialogue' : 'X', 'summary' : "y"})

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast = False)
model =  AutoModelForSeq2SeqLM.from_pretrained(model_id).to(device)



In [8]:
import langchain

X = samples_to_tune_on[0]['X']

pt = langchain.PromptTemplate.from_template("Conversation: {X}\nSummary:")

print(pt.format_prompt(X = X).to_string())

Conversation: Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-)
Summary:


In [9]:
import evaluate

rouge_metric = evaluate.load("rouge")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]
    

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def get_rouge_score(y_true: List, y_pred: List):
    preds, gts = postprocess_text(preds=y_pred, labels=y_true)

    result = rouge_metric.compute(predictions=preds, references=gts, use_stemmer=True)
    return result['rouge2']

In [10]:
from llmsearch.tuner import Tuner
from llmsearch.utils.mem_utils import get_total_available_ram, get_gpu_information
from llmsearch.utils.logging_utils import set_verbosity_info, set_verbosity_debug
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

seed = 42

set_verbosity_info()


In [11]:
tuner_ob = Tuner(model = model,tokenizer = tokenizer,dataset = samples_to_tune_on,
                 device = device, batch_size = 512,
                 tokenizer_encoding_kwargs={'padding': True, 'truncation': True, 'max_length': 512},
                 tokenizer_decoding_kwargs = {'skip_special_tokens' : True,  'spaces_between_special_tokens' : False}, 
                 scorer = get_rouge_score, prompt_template = pt, is_encoder_decoder = True, seed = seed, column_mapping = {"text_column_name": "X", "label_column_name": "y"})


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [12]:
len(samples_to_tune_on['X'])

1

In [13]:
1

1

In [ ]:
# Earlier
from llmsearch.utils.model_utils import seed_everything


"""
parameters and how they affect do_sample == False
1. temperature - output does not change - greedy decoding
2. top_k - output does not change - greedy decoding
3. repetition_penalty - output changes
4. no_repeat_ngram_size - output changes
"""

# seed_everything(seed)

initial_generation_params1 = {
    'max_new_tokens' : 120,
#     'repetition_penalty'  : 0.6,
#     'repetition_penalty_range'  : 5,
#     'temperature' : 0.7,
    'do_sample' : True,
    'generation_seed' : 42,
    'mirostat_mode': 2,
    'mirostat_tau' : 5,
#     'top_a' : 0.1,
}
score, outputs1 = tuner_ob.get_score(initial_generation_params1)



2023-09-03 18:23:24.206 - llmsearch.utils.mem_utils:145 - INFO - Starting inference with generation parameters - {'max_new_tokens': 120, 'do_sample': True, 'generation_seed': 42, 'mirostat_mode': 2, 'mirostat_tau': 5}
2023-09-03 18:23:24.207 - llmsearch.utils.mem_utils:149 - INFO - Performing inference with batch_size - 512
2023-09-03 18:23:24.208 - llmsearch.utils.model_utils:101 - INFO - Detected generation type - Sampling


  0%|          | 0/1 [00:00<?, ?it/s]

here - 2


/Users/praful932/Library/Caches/pypoetry/virtualenvs/bhaasha-VBcDHvMD-py3.8/lib/python3.8/site-packages/transformers/generation/utils.py:719: UserWarning: MPS: no support for int64 repeats mask, casting it to int32 (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Repeat.mm:236.)
  input_ids = input_ids.repeat_interleave(expand_size, dim=0)


Iteration - 1
[21542, 16637, 24571, 24244, 13390]
sorted logits tensor sum - -580818.8125
[0.9838887453079224, 0.011557682417333126, 0.0011503249406814575, 0.00023728713858872652]
row logit tensor sum - 8.781220436096191
Break index i - 3
previous index - tensor([0], device='mps:0')
logit val - tensor([0.9872], device='mps:0')
obs surprise value - 0.018514678748829252
indices to remove - 516098713
sum value - 6.658950328826904


/Users/praful932/Library/Caches/pypoetry/virtualenvs/bhaasha-VBcDHvMD-py3.8/lib/python3.8/site-packages/transformers/generation/utils.py:2671: UserWarning: MPS: no support for int64 min/max ops, casting it to int32 (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/ReduceOps.mm:1271.)
  if unfinished_sequences.max() == 0:


Iteration - 2
[13635, 56, 11, 65, 19]
sorted logits tensor sum - -582465.75
[0.6717739105224609, 0.13499009609222412, 0.04175287485122681, 0.02784247137606144]
row logit tensor sum - -47.7884521484375
Break index i - 27
previous index - tensor([0], device='mps:0')
logit val - tensor([0.6907], device='mps:0')
obs surprise value - 0.5337700339256832
indices to remove - 516106620
sum value - 3.236032009124756
Iteration - 3
[5081, 7364, 128, 3, 8]
sorted logits tensor sum - -556477.8125
[0.7825827598571777, 0.073967345058918, 0.06892622262239456, 0.016024406999349594]
row logit tensor sum - -28.038368225097656
Break index i - 19
previous index - tensor([0], device='mps:0')
logit val - tensor([0.8048], device='mps:0')
obs surprise value - 0.31335414644792475
indices to remove - 516115174
sum value - 3.9045827388763428
Iteration - 4
[5, 11, 21, 28, 6]
sorted logits tensor sum - -631473.0
[0.39131960272789, 0.24724672734737396, 0.1132679358124733, 0.08391641825437546]
row logit tensor sum - -

In [ ]:
outputs1

In [15]:
outputs1

['Amanda baked cookies with Jerry. She has another brother. Amanda will bring her tomorrow.']

In [47]:
outputs1

['Amanda baked cookies. Jerry will bring Amanda tomorrow.',
 'Olivia is voting for Liberals.',
 "Kim is going to do a lot of stuff but she's procrastinating. Kim will move her ass and do everything. Tim recommends Pomodoro technique where she uses breaks for doing chores.",
 'Edward is in ove with Rachel.',
 "Sam is worried about rick saying something to her roommate. He told them that he didn't like being her roommate. Sam is worried about him.",
 'Neville got married on September 17. Wyatt will ask her wife to check on her wedding anniversary.',
 'Cassandra will check the homework for John in 20 minutes. John will check it for him in 20 minutes. Cassandra will go to his boss to help him. John will help him with his homework.',
 "James found a song on youtube. He doesn't like it. He listens to it everytime he listens to it.",
 'Noah quit his job and quit his job. Madison thinks Noah liked it.',
 'Matt and Agnes are going to the Georgian restaurant in Kazimierz on Saturday at 6 pm. Mat

In [ ]:
for y_t, y_p in zip(tuner_ob.dataset['y'], outputs1):
    print(y_p)
    print(y_t,'\n\n')

In [ ]:
print(outputs1)

In [ ]:
print(outputs1)

In [ ]:
hyp_param_grid = {
    "max_new_tokens": [120],
    "temperature": list(np.linspace(start=0.1, stop=1.0,num=10)),
    'top_k' : list(map(int,np.linspace(start=10, stop=50,num=5))),
    "top_p": [0.75, 0.8, 0.9, 1.0],
    'do_sample' : [True, False],
    'generation_seed' : [42],
    'repetition_penalty' : [1.0, 1.2],
    'no_repeat_ngram_size' : [0,2,3],
}

hyp_param_grid_2= {
    "max_new_tokens": [120],
    "temperature": list(np.linspace(start=0.1, stop=1.0,num=10000)),
    'top_k' : list(map(int,np.linspace(start=10, stop=50,num=5000))),
    "top_p": list(map(int,np.linspace(start=10, stop=50,num=5000))),
    'do_sample' : [True],
    'generation_seed' : [42],
    'num_beams' : [1],
#     'repetition_penalty' : [1.0, 1.2],
#     'no_repeat_ngram_size' : [0,2,3],
}

scorer = make_scorer(score_func=get_rouge_score, greater_is_better=True)


clf = RandomizedSearchCV(
    estimator=tuner_ob.estimator,
    param_distributions=hyp_param_grid_2,
    n_iter = 2,
    scoring=scorer,
    cv=5,
    random_state = 42,
    n_jobs=None,
)

"""
5 fold means, whole sample set of 100 examples will be split into 80:20 ratio
for each hyper_parameter set we have a model f(hyper_params)
    - we will evaluate this model and get the cross val score (test on each 20 samples 5 times, while training on the rest 80 each time)
    - we get the score on the quality of hyperparams by evaluating the model with the hyperparams on the unseen 1 fold
"""


In [ ]:
1 = 1/0

In [ ]:
clf.fit(X=tuner_ob.dataset["X"], y=tuner_ob.dataset["y"])

In [ ]:
clf.best_params_

In [ ]:
clf.best_estimator_

In [ ]:
clf.best_estimator_.set_params(**clf.best_params_).get_params()


In [ ]:
clf.fi

In [ ]:
from sklearn.base import clone

clone(1)

In [ ]:
dir(clf)

In [ ]:
clf.best_estimator_.get_params()

In [ ]:
clf.best_estimator_.get_params()

In [ ]:
clf.cv_results_

In [ ]:
clf.best_estimator_.get_params()